# Test 2

## Initialization 

In [ ]:
import os
import sys
import json
import openai

openai_config = json.load(open("openai.json", "r+"))
openai.api_key = openai_config["api_key"]
openai.organization = openai_config["organization_id"]

## Library

In [ ]:
# Conversation

# Message = { role: String, content: String }
# Messages = List<Message>

class ConversationTree:
    def __init__(self):
        self.kids = None

    def insert(self, node):
        ConversationNode.insert(self.kids, node)

    def load(filename):
        return ConversationTree.fromJSON(json.load(open(filename, "r+")))

    def save(self, filename):
        json.dump(self.toJSON(), open(filename, "w+"))

    # obj: { name: String, node: Obj }
    def fromJSON(obj):
        convo = ConversationTree()
        convo.name = obj['name']
        convo.kids = [ConversationNode.fromJSON(node) for node in obj['kids']]
        return convo

    def toJSON(self):
        return {
            'name': self.name,
            'kids': self.kids.toJSON()
        }

# ConversationNodeValue = { case: 'Nil' }
#                       | { case: 'Message', message: Message }
#                       | { case: 'Query', message: Message | None }


class ConversationNode:
    def __init__(self, value):
        self.value = value
        self.kids = []

    def Nil():
        return ConversationNode('Nil')

    def Message(role, content):
        node = ConversationNode({
            'case': 'Message',
            'content': {'role': role, 'content': content}
        })
        return node

    def Query():
        node = ConversationNode({
            'case': 'Query',
            'content': None
        })
        return node

    # values: List<ConversationNodeValues>
    # modifies kids
    def insert(kids, values):
        if len(values) == 0:
            return
        else:
            value = values.pop(0)
            if value.case == 'Message':
                for i in range(len(kids)):
                    kid = kids[i]
                    assert kid.case == 'Message'
                    if kid.value.content == value.content:
                        # remove this kid from kids (put at end of kids later)
                        kids.pop(i)
                        # recurse
                        ConversationNode.insert(kid.kids, values)
                        # put kid at the end of kids
                        kids.append(kid)
                        return
                kids.append(ConversationNode.fromList(values))
                return

            elif value.case == 'Query':
                if value.message is None:
                    # just append to end of kids then
                    values.insert(0, value)
                    kids.append(ConversationNode.fromList(values))
                    return

                # assume the value.message is not None

                for i in range(len(kids)):
                    kid = kids[i]
                    assert kid.case == 'Query'
                    if kid.value.message is None:
                        # remove this kid from kids (put at end of kids later)
                        kids.pop(i)
                        # replace with new query
                        kid.value = value
                        # recurse
                        ConversationNode.insert(kid.kids, values)
                        # put kid at end of kids
                        kids.append(kid)
                        return
                    else:
                        if kid.value.message.content == value.message.content:
                            # match
                            ConversationNode.insert(kid.kids, values)
                            return
                        else:
                            # doesn't match
                            pass
                # if no matches or special cases, then just append to kids
                values.insert(0, value)
                kids.append(ConversationNode.fromList(values))
                return

    # obj: { case: 'Nil' }
    #    | { case: 'Message', message: Message, kids: Obj[] }
    #    | { case: 'Query', message: Message | None, kids: Obj[] }

    def fromJSON(obj):
        node = ConversationNode()
        node.case = obj['case']
        if node.case == 'Nil':
            pass
        elif node.case == 'Message':
            node.message = obj['message']
            node.kids = map(ConversationNode.fromJSON, obj['kids'])
        elif node.case == 'Query':
            node.message = obj['message']
            node.kids = map(ConversationNode.fromJSON, obj['kids'])
        return node

    def toJSON(self):
        if self.case == 'Nil':
            return {
                'case': 'Nil'
            }
        elif self.case == 'Message':
            return {
                'case': 'Message',
                'message': self.message,
                'kids': [node.toJSON for node in self.kids]
            }
        elif self.case == 'Query':
            return {
                'case': 'Message',
                'message': self.message,
                'kids': [node.toJSON for node in self.kids]
            }

    def fromList(self, values):
        if len(values) == 0:
            return ConversationNode.Nil()
        else:
            value0 = values.pop(0)
            node0 = ConversationNode(value0)
            node = node0
            for value in values:
                kid = ConversationNode(value)
                node.kids.append(kid)
                node = kid
            return node0


## Experiments